In [1]:
import geopandas as gpd

link="https://github.com/chorrillos/preprocesamiento/raw/main/distritos/DistritosMap.zip"
selecDisMap=gpd.read_file(link)

In [2]:
import pandas as pd
link2="https://github.com/chorrillos/spatial/raw/main/consolidado.variables.seleccionadas.xlsx"
datadis=pd.read_excel(link2)

In [3]:
selecDisMap[selecDisMap.DISTRITO.str.contains('OCTUBRE')]

,DEPARTAMEN,PROVINCIA,DISTRITO,INSTITUCIO,geometry
1800,PIURA,PIURA,VEINTISEIS DE OCTUBRE,IGN,"POLYGON ((-80.65416 -5.17733, -80.64678 -5.182..."


In [4]:
datadis['Distrito'][datadis.Distrito.str.contains('OCTUBRE')]="VEINTISEIS DE OCTUBRE"

/var/folders/2n/bkfhfqq16r78g3hf7pdj56y40000gn/T/ipykernel_27969/1185489563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datadis['Distrito'][datadis.Distrito.str.contains('OCTUBRE')]="VEINTISEIS DE OCTUBRE"


In [5]:
#todo mayuscula sin blanco delante ni cola 


datadis[['PROVINCIA','Distrito']]=datadis[['PROVINCIA','Distrito']].apply(lambda x: x.str.upper().str.strip())
selecDisMap[['PROVINCIA','DISTRITO']]=selecDisMap[['PROVINCIA','DISTRITO']].apply(lambda x: x.str.upper().str.strip())
import unidecode

# concatenando
datadis['provDist']=["+".join(pd) for pd in zip (datadis.PROVINCIA,datadis.Distrito)]
selecDisMap['provDist']=["+".join(pd) for pd in zip (selecDisMap.PROVINCIA,selecDisMap.DISTRITO)]


# eliminando simbolos 
datadis.provDist=[unidecode.unidecode(dist) for dist in datadis.provDist]
selecDisMap.provDist=[unidecode.unidecode(dist) for dist in selecDisMap.provDist]

# reemplazando guiones y espacios multiples por espacio simple
datadis.provDist=datadis.provDist.str.replace("\-|\_|\s+"," ",regex=True)
selecDisMap.provDist=selecDisMap.provDist.str.replace("\-|\_|\s+"," ",regex=True)

In [6]:
# valores sin march
sinmatchDATA=list(set(datadis.provDist)- set(selecDisMap.provDist))
sinmatchMAP=list(set(selecDisMap.provDist)-set(datadis.provDist) )

In [7]:
len(sinmatchDATA), len(sinmatchMAP)

(13, 13)

In [8]:
from thefuzz import process
[(dis,process.extractOne(dis,sinmatchMAP)) for dis in sorted(sinmatchDATA)]

[('ANGARAES+HUAYLLAY GRANDE', ('ANGARAES+HUALLAY GRANDE', 98)),
 ('ATALAYA+RAIMONDI', ('ATALAYA+RAYMONDI', 94)),
 ('AYMARAES+CARAYBAMBA', ('AYMARAES+CARAIBAMBA', 95)),
 ('AYMARAES+HUAYLLO', ('AYMARAES+IHUAYLLO', 97)),
 ('CALLAO+CARMEN DE LA LEGUA', ('CALLAO+CARMEN DE LA LEGUA REYNOSO', 95)),
 ('CHUPACA+SAN JUAN DE ISCOS', ('CHUPACA+SAN JUAN DE YSCOS', 96)),
 ('GRAN CHIMU+COMPIN', ('GRAN CHIMU+MARMOT', 71)),
 ('LIMA+PUEBLO LIBRE', ('LIMA+MAGDALENA VIEJA', 49)),
 ('MAYNAS+PUTUMAYO', ('PUTUMAYO+PUTUMAYO', 95)),
 ('MAYNAS+ROSA PANDURO', ('PUTUMAYO+ROSA PANDURO', 80)),
 ('MAYNAS+TENIENTE MANUEL CLAVERO', ('PUTUMAYO+TENIENTE MANUEL CLAVERO', 87)),
 ('MAYNAS+YAGUAS', ('PUTUMAYO+YAGUAS', 71)),
 ('SAN MARTIN+PAPAPLAYA', ('SAN MARTIN+PAPA PLAYA', 98))]

In [9]:
cambiosDataDis={dis:process.extractOne(dis,sinmatchMAP)[0] for dis in sorted(sinmatchDATA)}
cambiosDataDis

{'ANGARAES+HUAYLLAY GRANDE': 'ANGARAES+HUALLAY GRANDE',
 'ATALAYA+RAIMONDI': 'ATALAYA+RAYMONDI',
 'AYMARAES+CARAYBAMBA': 'AYMARAES+CARAIBAMBA',
 'AYMARAES+HUAYLLO': 'AYMARAES+IHUAYLLO',
 'CALLAO+CARMEN DE LA LEGUA': 'CALLAO+CARMEN DE LA LEGUA REYNOSO',
 'CHUPACA+SAN JUAN DE ISCOS': 'CHUPACA+SAN JUAN DE YSCOS',
 'GRAN CHIMU+COMPIN': 'GRAN CHIMU+MARMOT',
 'LIMA+PUEBLO LIBRE': 'LIMA+MAGDALENA VIEJA',
 'MAYNAS+PUTUMAYO': 'PUTUMAYO+PUTUMAYO',
 'MAYNAS+ROSA PANDURO': 'PUTUMAYO+ROSA PANDURO',
 'MAYNAS+TENIENTE MANUEL CLAVERO': 'PUTUMAYO+TENIENTE MANUEL CLAVERO',
 'MAYNAS+YAGUAS': 'PUTUMAYO+YAGUAS',
 'SAN MARTIN+PAPAPLAYA': 'SAN MARTIN+PAPA PLAYA'}

In [10]:
datadis.provDist.replace(cambiosDataDis,inplace=True)

In [11]:
# anadiendo CSV al MAPA:
selecDisMap=selecDisMap.merge(datadis, on='provDist')

In [12]:
selecDisMap.columns

Index(['DEPARTAMEN', 'PROVINCIA_x', 'DISTRITO', 'INSTITUCIO', 'geometry',
       'provDist', 'Ubigeo', 'DEPARTAMENTO', 'PROVINCIA_y', 'Distrito', 'Zona',
       'Latitud', 'Longitud', 'cobertura.cell', 'no.cobertura.cell',
       'Educ.sec.comp', 'Educ.sec.no.comp', 'esp.vid.nacer', 'Ing.fam.p.cap',
       'NBI.2017', 'NBI.2017.c', 'Pobreza', 'No.Pobreza'],
      dtype='object')

In [13]:
sacar=['Ubigeo', 'DEPARTAMEN', 'PROVINCIA_x','INSTITUCIO', 'Distrito']
selecDisMap.drop(sacar,axis=1,inplace=True)

In [14]:
selecDisMap.rename(columns={'PROVINCIA_y':"PROVINCIA"},inplace=True)

In [15]:
selecDisMap

,DISTRITO,geometry,provDist,DEPARTAMENTO,PROVINCIA,Zona,Latitud,Longitud,cobertura.cell,no.cobertura.cell,Educ.sec.comp,Educ.sec.no.comp,esp.vid.nacer,Ing.fam.p.cap,NBI.2017,NBI.2017.c,Pobreza,No.Pobreza
0,CORONEL GREGORIO ALBARRACIN LANCHIPA,"POLYGON ((-70.17413 -18.12896, -70.17461 -18.1...",TACNA+CORONEL GREGORIO ALBARRACIN LANCHIPA,TACNA,TACNA,X,-18.0408,-70.2542,47.058824,52.941176,71.178389,28.821611,77.073616,892.303097,15.8,84.2,18.736677,81.263323
1,POCOLLAY,"POLYGON ((-69.93475 -17.92557, -69.90467 -17.9...",TACNA+POCOLLAY,TACNA,TACNA,X,-17.9964,-70.2197,64.705882,35.294118,75.825743,24.174257,77.267808,1152.606145,16.1,83.9,13.404542,86.595458
2,CALANA,"POLYGON ((-70.11604 -17.91106, -70.11457 -17.9...",TACNA+CALANA,TACNA,TACNA,X,-17.9406,-70.1825,59.574468,40.425532,77.829717,22.170283,75.922728,832.802135,15.9,84.1,7.304348,92.695652
3,TACNA,"POLYGON ((-70.31490 -17.94498, -70.30682 -17.9...",TACNA+TACNA,TACNA,TACNA,X,-18.0100,-70.2478,33.333333,66.666667,75.491958,24.508042,76.670661,1371.819928,7.4,92.6,7.691274,92.308726
4,SAMA,"POLYGON ((-70.42497 -17.88934, -70.48022 -17.9...",TACNA+SAMA,TACNA,TACNA,X,-17.8586,-70.5731,61.111111,38.888889,70.500250,29.499750,74.249879,897.568256,52.4,47.6,10.643201,89.356799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,CIENEGUILLA,"POLYGON ((-76.72917 -11.99948, -76.72596 -11.9...",LIMA+CIENEGUILLA,LIMA,LIMA,ZNG,-12.1178,-76.8125,100.000000,0.000000,69.643266,30.356734,77.288096,1265.041999,24.7,75.3,17.529394,82.470606
1870,ANCON,"POLYGON ((-77.06501 -11.57523, -77.06499 -11.5...",LIMA+ANCON,LIMA,LIMA,ZNG,-11.7764,-77.1703,100.000000,0.000000,72.117976,27.882024,77.659716,1392.956247,21.7,78.3,19.246935,80.753065
1871,CARABAYLLO,"POLYGON ((-76.89892 -11.72963, -76.89768 -11.7...",LIMA+CARABAYLLO,LIMA,LIMA,ZNG,-11.8583,-77.0419,82.500000,17.500000,74.114567,25.885433,78.849674,1246.479589,20.0,80.0,19.176021,80.823979
1872,MI PERU,"POLYGON ((-77.11491 -11.84189, -77.11450 -11.8...",CALLAO+MI PERU,CALLAO,CALLAO,X,-11.8550,-77.1250,100.000000,0.000000,67.812626,32.187374,81.186620,1241.061299,17.1,82.9,23.941494,76.058506


In [16]:
selecDisMap.to_file("selecDisMap.gpkg", layer='DISTRITO', driver="GPKG")